In [84]:
import numpy as np
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
from sklearn.metrics import f1_score, precision_score, precision_recall_curve, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from scipy.stats.mstats import winsorize


In [85]:
#Import the data 
# read all tables 
customer_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/customers_final.csv')
engagement_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/engagements_final.csv')
marketing_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/marketing_final.csv')
transactions_final = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/transactions_final.csv')

In [86]:
# deal with gender null 
# deal with age null
customer_final.dropna(subset=['gender', 'age'], inplace=True)

In [87]:
customer_final['join_date'] = pd.to_datetime(customer_final['join_date'])
customer_final['last_purchase_date'] = pd.to_datetime(customer_final['last_purchase_date'])
transactions_final['transaction_date'] = pd.to_datetime(transactions_final['transaction_date'])
marketing_final['campaign_date'] = pd.to_datetime(marketing_final['campaign_date'])

In [88]:
#creating CLV dataset 
clv_final = transactions_final.groupby('customer_id')['transaction_amount'].sum().reset_index()
clv_final.rename(columns={'transaction_amount': 'clv'}, inplace=True) 


In [89]:
transactions_agg = transactions_final.groupby('customer_id').aggregate({'transaction_id':'count','transaction_amount':'sum'})

In [90]:
marketing_agg = marketing_final.groupby('customer_id').size().reset_index(name='number_of_campaigns')
marketing_agg.set_index('customer_id', inplace=True)


In [91]:
engagement_final.set_index('customer_id', inplace=True)
customer_final.set_index('customer_id', inplace=True)

In [92]:
clv_final.set_index('customer_id', inplace=True)

In [93]:
joint_dataa = customer_final.join(engagement_final).join(transactions_agg).join(marketing_agg).join(clv_final)

In [94]:
print(joint_dataa.isnull().sum())

join_date                  0
last_purchase_date         0
age                        0
gender                     0
location                   0
number_of_site_visits      0
number_of_emails_opened    0
number_of_clicks           0
transaction_id             0
transaction_amount         0
number_of_campaigns        0
clv                        0
dtype: int64


In [95]:
joint_dataa

,join_date,last_purchase_date,age,gender,location,number_of_site_visits,number_of_emails_opened,number_of_clicks,transaction_id,transaction_amount,number_of_campaigns,clv
customer_id,,,,,,,,,,,,
1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,10,15,1,6,3509.48,4,3509.48
4,2022-01-01,2022-09-01,29.0,Male,Grossstad,110,30,17,20,7874.68,4,7874.68
6,2022-01-01,2023-02-10,55.0,Female,North Richardfort,74,59,7,17,10254.44,3,10254.44
8,2023-02-09,2023-08-09,68.0,Male,Marquezton,60,22,7,15,11710.62,3,11710.62
9,2022-01-24,2023-08-15,68.0,Female,West Franciscobury,49,43,21,22,10382.96,2,10382.96
...,...,...,...,...,...,...,...,...,...,...,...,...
9996,2022-12-16,2023-08-13,42.0,Female,Johnstonborough,119,47,16,10,5498.20,3,5498.20
9997,2022-07-09,2023-01-25,26.0,Male,Jessicamouth,3,33,14,12,5848.30,1,5848.30
9998,2023-09-17,2024-01-30,39.0,Male,New John,53,17,5,3,3503.13,1,3503.13


In [96]:
joint_dataa['customer_lifetime'] = (joint_dataa['last_purchase_date'] - joint_dataa['join_date']).dt.days

In [97]:
joint_dataa

,join_date,last_purchase_date,age,gender,location,number_of_site_visits,number_of_emails_opened,number_of_clicks,transaction_id,transaction_amount,number_of_campaigns,clv,customer_lifetime
customer_id,,,,,,,,,,,,,
1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,10,15,1,6,3509.48,4,3509.48,118
4,2022-01-01,2022-09-01,29.0,Male,Grossstad,110,30,17,20,7874.68,4,7874.68,243
6,2022-01-01,2023-02-10,55.0,Female,North Richardfort,74,59,7,17,10254.44,3,10254.44,405
8,2023-02-09,2023-08-09,68.0,Male,Marquezton,60,22,7,15,11710.62,3,11710.62,181
9,2022-01-24,2023-08-15,68.0,Female,West Franciscobury,49,43,21,22,10382.96,2,10382.96,568
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,2022-12-16,2023-08-13,42.0,Female,Johnstonborough,119,47,16,10,5498.20,3,5498.20,240
9997,2022-07-09,2023-01-25,26.0,Male,Jessicamouth,3,33,14,12,5848.30,1,5848.30,200
9998,2023-09-17,2024-01-30,39.0,Male,New John,53,17,5,3,3503.13,1,3503.13,135


In [98]:
# Normalize CLV by customer lifetime
joint_dataa['normalized_clv'] = joint_dataa['clv'] / joint_dataa['customer_lifetime']

# Winsorize the customer_lifetime to handle zeros and extreme values
joint_dataa['customer_lifetime'] = winsorize(joint_dataa['customer_lifetime'], limits=[0.01, 0.01])


In [99]:
# Normalize CLV by customer lifetime, avoid division by zero


In [100]:
joint_dataa

,join_date,last_purchase_date,age,gender,location,number_of_site_visits,number_of_emails_opened,number_of_clicks,transaction_id,transaction_amount,number_of_campaigns,clv,customer_lifetime,normalized_clv
customer_id,,,,,,,,,,,,,,
1,2023-11-20,2024-03-17,56.0,Female,North Shannonbury,10,15,1,6,3509.48,4,3509.48,118,29.741356
4,2022-01-01,2022-09-01,29.0,Male,Grossstad,110,30,17,20,7874.68,4,7874.68,243,32.406091
6,2022-01-01,2023-02-10,55.0,Female,North Richardfort,74,59,7,17,10254.44,3,10254.44,405,25.319605
8,2023-02-09,2023-08-09,68.0,Male,Marquezton,60,22,7,15,11710.62,3,11710.62,181,64.699558
9,2022-01-24,2023-08-15,68.0,Female,West Franciscobury,49,43,21,22,10382.96,2,10382.96,568,18.279859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,2022-12-16,2023-08-13,42.0,Female,Johnstonborough,119,47,16,10,5498.20,3,5498.20,240,22.909167
9997,2022-07-09,2023-01-25,26.0,Male,Jessicamouth,3,33,14,12,5848.30,1,5848.30,200,29.241500
9998,2023-09-17,2024-01-30,39.0,Male,New John,53,17,5,3,3503.13,1,3503.13,135,25.949111


In [101]:
zero_lifetime_customers = joint_dataa[joint_dataa['customer_lifetime'] == 0]

# Display the count
zero_lifetime_count = zero_lifetime_customers.shape[0]
zero_lifetime_count

0

if they have a zero lifetime we will not be able to divide it likely means they joined on the day of thier last purcase (at same time) so i have a few options assuming zero lifetime =1 because they could join at 1159pm and order at 0001 next day which is effectively the same

winsorize to .01 

In [102]:
# Feature selection
features = ['age', 'gender','number_of_site_visits', 'number_of_emails_opened',
            'number_of_clicks', 'number_of_campaigns', 'transaction_id']
target = 'normalized_clv'



In [103]:
features

['age',
 'gender',
 'number_of_site_visits',
 'number_of_emails_opened',
 'number_of_clicks',
 'number_of_campaigns',
 'transaction_id']

In [104]:
# One-hot encode the 'gender' column
joint_dataa = pd.get_dummies(joint_dataa, columns=['gender'], drop_first=True)

In [105]:
joint_dataa

,join_date,last_purchase_date,age,location,number_of_site_visits,number_of_emails_opened,number_of_clicks,transaction_id,transaction_amount,number_of_campaigns,clv,customer_lifetime,normalized_clv,gender_Male
customer_id,,,,,,,,,,,,,,
1,2023-11-20,2024-03-17,56.0,North Shannonbury,10,15,1,6,3509.48,4,3509.48,118,29.741356,False
4,2022-01-01,2022-09-01,29.0,Grossstad,110,30,17,20,7874.68,4,7874.68,243,32.406091,True
6,2022-01-01,2023-02-10,55.0,North Richardfort,74,59,7,17,10254.44,3,10254.44,405,25.319605,False
8,2023-02-09,2023-08-09,68.0,Marquezton,60,22,7,15,11710.62,3,11710.62,181,64.699558,True
9,2022-01-24,2023-08-15,68.0,West Franciscobury,49,43,21,22,10382.96,2,10382.96,568,18.279859,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,2022-12-16,2023-08-13,42.0,Johnstonborough,119,47,16,10,5498.20,3,5498.20,240,22.909167,False
9997,2022-07-09,2023-01-25,26.0,Jessicamouth,3,33,14,12,5848.30,1,5848.30,200,29.241500,True
9998,2023-09-17,2024-01-30,39.0,New John,53,17,5,3,3503.13,1,3503.13,135,25.949111,True


In [106]:
# Convert to integer if necessary
joint_dataa['gender_Male'] = joint_dataa['gender_Male'].astype(int)

In [107]:
joint_dataa

,join_date,last_purchase_date,age,location,number_of_site_visits,number_of_emails_opened,number_of_clicks,transaction_id,transaction_amount,number_of_campaigns,clv,customer_lifetime,normalized_clv,gender_Male
customer_id,,,,,,,,,,,,,,
1,2023-11-20,2024-03-17,56.0,North Shannonbury,10,15,1,6,3509.48,4,3509.48,118,29.741356,0
4,2022-01-01,2022-09-01,29.0,Grossstad,110,30,17,20,7874.68,4,7874.68,243,32.406091,1
6,2022-01-01,2023-02-10,55.0,North Richardfort,74,59,7,17,10254.44,3,10254.44,405,25.319605,0
8,2023-02-09,2023-08-09,68.0,Marquezton,60,22,7,15,11710.62,3,11710.62,181,64.699558,1
9,2022-01-24,2023-08-15,68.0,West Franciscobury,49,43,21,22,10382.96,2,10382.96,568,18.279859,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,2022-12-16,2023-08-13,42.0,Johnstonborough,119,47,16,10,5498.20,3,5498.20,240,22.909167,0
9997,2022-07-09,2023-01-25,26.0,Jessicamouth,3,33,14,12,5848.30,1,5848.30,200,29.241500,1
9998,2023-09-17,2024-01-30,39.0,New John,53,17,5,3,3503.13,1,3503.13,135,25.949111,1


In [108]:
# Adjusted feature selection
features = ['age', 'number_of_site_visits', 'number_of_emails_opened',
            'number_of_clicks', 'number_of_campaigns', 'transaction_id', 'gender_Male']
target = 'normalized_clv'

In [109]:
# Scale numerical features
scaler = MinMaxScaler()
joint_dataa[features] = scaler.fit_transform(joint_dataa[features])

In [110]:
# Define the feature set and the target variable
X = joint_dataa[features]
y = joint_dataa[target]

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [112]:
X_test.head()

,age,number_of_site_visits,number_of_emails_opened,number_of_clicks,number_of_campaigns,transaction_id,gender_Male
customer_id,,,,,,,
8810,0.313725,0.433803,0.099338,0.312057,0.000000,0.875000,1.0
8791,0.784314,0.012676,0.016556,0.014184,0.333333,0.583333,1.0
3032,0.549020,0.069014,0.039735,0.007092,0.333333,0.875000,0.0
1295,0.000000,0.674648,0.261589,0.333333,1.000000,0.250000,0.0
1020,0.000000,0.159155,0.003311,0.113475,1.000000,0.125000,0.0


trying linear regression 

In [114]:
# Model selection and training
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

ValueError: Input y contains infinity or a value too large for dtype('float64').